# 3日目 - 会話AI - aka chatbot！

この Jupyter Notebook（week2/day3.ipynb）の内容の概要は以下の通り。

主な流れは以下の通りです：

1. **APIキーの取得と環境変数の設定**
   - `.env`ファイルからOpenAI、Anthropic、GoogleのAPIキーを読み込み、キーが存在するか確認します。

2. **OpenAI APIの初期化**
   - `OpenAI()`オブジェクトを作成し、利用するモデル（例: gpt-4o-mini）を指定します。

3. **システムメッセージの定義**
   - アシスタントのキャラクターや振る舞いを決める「system_message」を設定します。例として「あなたは衣料品店の親切なアシスタントです。帽子や他の商品を勧めてください。」など。

4. **チャット関数の実装**  
   - `chat(message, history)`という関数を作成。  
   - `system_message`、これまでの会話履歴（history）、新しいユーザーからのメッセージをまとめてOpenAI APIに送り、ストリームで応答を生成します。

5. **GradioによるUI構築**  
   - `gr.ChatInterface(fn=chat, type="messages").launch()`で、チャットボットのWebインターフェースを立ち上げます。

6. **システムメッセージやチャット関数のカスタマイズ**  
   - 商品のセール情報や、特定の質問（例：ベルトの問い合わせ）への対応など、system_messageやchat関数の内容を条件に応じて変更する例も紹介されています。

7. **ビジネス応用例の紹介**  
   - 会話型AIをビジネスにどう活用できるか、システムプロンプトを工夫して自分のビジネス向けプロトタイプを作ることを推奨しています。

要約すると、このノートブックは「Gradio＋OpenAI API」を使った会話型AI（チャットボット）の実装、カスタマイズ方法、そしてビジネス応用のアイデアを学ぶためのものです。

In [1]:
# import

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# .envというファイルに環境変数をロードします
# キープレフィックスを印刷して、デバッグに役立ちます

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI APIキーが存在し、開始します {openai_api_key[:8]}")
else:
    print("Openai APIキーが設定されていません")
    
if anthropic_api_key:
    print(f"Anthropic APIキーが存在し、開始します {anthropic_api_key[:7]}")
else:
    print("人為的APIキーが設定されていません")

if google_api_key:
    print(f"Google APIキーが存在し、開始します {google_api_key[:8]}")
else:
    print("Google APIキーが設定されていません")

OpenAI APIキーが存在し、開始します sk-proj-
人為的APIキーが設定されていません
Google APIキーが設定されていません


In [3]:
# 初期化

openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [4]:
system_message = "あなたは役に立つアシスタントです"

# これを読んでください！ビデオからの変更：

ビデオでは、次のような関数を書く必要がある方法を説明します。

`chat(message, history)`

これは、特定の形式で`history`を受け取ることを期待しています。

これは、OpenAIを呼び出す前にOpenAI形式にマッピングする必要があります。

```json
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

しかし、Gradioはアップグレードされました！

`history`をOpenAI形式のまま渡すようになり、OpenAIに直接送信できるようになりました。

これで私たちの仕事は簡単になりました！

関数 `chat(message, history)` を書きます。:  
**message** は使用するプロンプトです  
**history** は、過去の会話であり、OpenAI形式です  

システムメッセージ、履歴、最新のメッセージを組み合わせて、OpenAIをコールします。

In [5]:
# 私のビデオよりもシンプル - OpenAIを呼び出すこの関数を簡単に作成できます
# OpenAIへの入力を準備するためのコードが1行になりました！

# 学生のOctavio O.は、これはClaudeにとってそれほど簡単ではないことを指摘しました
# Claudeを処理するコミュニティ委員会「Gradio_issue_with_claude」における優れた貢献を参照してください。

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    # コレはログ
    print("「history」は次のとおりです。")
    print(history)
    print("そして「messages」は次のとおりです。")
    print(messages)

    # レスポンスをストリーミング
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

## そして、Gradio's Magicを入力してください！

In [6]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


「history」は次のとおりです。
[]
そして「messages」は次のとおりです。
[{'role': 'system', 'content': 'あなたは役に立つアシスタントです'}, {'role': 'user', 'content': 'こんにちは'}]
「history」は次のとおりです。
[{'role': 'user', 'metadata': None, 'content': 'こんにちは', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'こんにちは！どのようにお手伝いできるでしょうか？', 'options': None}]
そして「messages」は次のとおりです。
[{'role': 'system', 'content': 'あなたは役に立つアシスタントです'}, {'role': 'user', 'metadata': None, 'content': 'こんにちは', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'こんにちは！どのようにお手伝いできるでしょうか？', 'options': None}, {'role': 'user', 'content': 'ネクタイを買いたい'}]
「history」は次のとおりです。
[{'role': 'user', 'metadata': None, 'content': 'こんにちは', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'こんにちは！どのようにお手伝いできるでしょうか？', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'ネクタイを買いたい', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'ネクタイを選ぶ際に考慮するポイントをいくつかご紹介しますね：\n\n1. **用途**: ビジネス、フォーマル、カジュアルなど、どの場面で

In [7]:
system_message = "あなたは衣料品店で親切な店員として働いています。\
お客様にセール品を試着するよう、優しく勧めてみましょう。\
帽子は60%オフ、その他のほとんどの商品は50%オフです。\
例えば、お客様が「帽子を買おうと思っているのですが」と尋ねたら、\
「ありがとうございます。帽子はたくさんございます。セール品もいくつかありますよ」\
などと答えることができます。お客様が何を買おうか迷っている場合は、帽子の購入を勧めましょう。"

In [8]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    # コレはログ
    print("「history」は次のとおりです。")
    print(history)
    print("そして「messages」は次のとおりです。")
    print(messages)
    
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [9]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


「history」は次のとおりです。
[]
そして「messages」は次のとおりです。
[{'role': 'system', 'content': 'あなたは衣料品店で親切な店員として働いています。お客様にセール品を試着するよう、優しく勧めてみましょう。帽子は60%オフ、その他のほとんどの商品は50%オフです。例えば、お客様が「帽子を買おうと思っているのですが」と尋ねたら、「ありがとうございます。帽子はたくさんございます。セール品もいくつかありますよ」などと答えることができます。お客様が何を買おうか迷っている場合は、帽子の購入を勧めましょう。'}, {'role': 'user', 'content': 'こんにちは'}]
「history」は次のとおりです。
[{'role': 'user', 'metadata': None, 'content': 'こんにちは', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'こんにちは！いらっしゃいませ。今日はどのようなものをお探しですか？', 'options': None}]
そして「messages」は次のとおりです。
[{'role': 'system', 'content': 'あなたは衣料品店で親切な店員として働いています。お客様にセール品を試着するよう、優しく勧めてみましょう。帽子は60%オフ、その他のほとんどの商品は50%オフです。例えば、お客様が「帽子を買おうと思っているのですが」と尋ねたら、「ありがとうございます。帽子はたくさんございます。セール品もいくつかありますよ」などと答えることができます。お客様が何を買おうか迷っている場合は、帽子の購入を勧めましょう。'}, {'role': 'user', 'metadata': None, 'content': 'こんにちは', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'こんにちは！いらっしゃいませ。今日はどのようなものをお探しですか？', 'options': None}, {'role': 'user', 'content': '靴を買いたい'}]
「histor

In [10]:
system_message += "\nお客様が靴を尋ねた場合は、靴は本日セールではないと答え、帽子も見るようにお客様に伝えなくてはならない。"

In [11]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


「history」は次のとおりです。
[]
そして「messages」は次のとおりです。
[{'role': 'system', 'content': 'あなたは衣料品店で親切な店員として働いています。お客様にセール品を試着するよう、優しく勧めてみましょう。帽子は60%オフ、その他のほとんどの商品は50%オフです。例えば、お客様が「帽子を買おうと思っているのですが」と尋ねたら、「ありがとうございます。帽子はたくさんございます。セール品もいくつかありますよ」などと答えることができます。お客様が何を買おうか迷っている場合は、帽子の購入を勧めましょう。\nお客様が靴を尋ねた場合は、靴は本日セールではないと答え、帽子も見るようにお客様に伝えなくてはならない。'}, {'role': 'user', 'content': 'こんにちは'}]
「history」は次のとおりです。
[{'role': 'user', 'metadata': None, 'content': 'こんにちは', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'こんにちは！いらっしゃいませ。何かお探しのものはありますか？お気軽にお声がけください。', 'options': None}]
そして「messages」は次のとおりです。
[{'role': 'system', 'content': 'あなたは衣料品店で親切な店員として働いています。お客様にセール品を試着するよう、優しく勧めてみましょう。帽子は60%オフ、その他のほとんどの商品は50%オフです。例えば、お客様が「帽子を買おうと思っているのですが」と尋ねたら、「ありがとうございます。帽子はたくさんございます。セール品もいくつかありますよ」などと答えることができます。お客様が何を買おうか迷っている場合は、帽子の購入を勧めましょう。\nお客様が靴を尋ねた場合は、靴は本日セールではないと答え、帽子も見るようにお客様に伝えなくてはならない。'}, {'role': 'user', 'metadata': None, 'content': 'こんにちは', 'options': None}, {'role': 'assistant', 'metadata': No

In [12]:
# 学生Gabor M.によって見事に識別されたこの関数のバグを修正しました！ また、この機能の構造も改善しました

def chat(message, history):

    relevant_system_message = system_message
    if 'ベルト' in message:
        relevant_system_message += " 当店ではベルトは販売しておりませんので、ベルトを求められた場合は、必ず他の販売商品をお伝えください。"
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    # コレはログ
    print("「history」は次のとおりです。")
    print(history)
    print("そして「messages」は次のとおりです。")
    print(messages)
    
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [13]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


「history」は次のとおりです。
[]
そして「messages」は次のとおりです。
[{'role': 'system', 'content': 'あなたは衣料品店で親切な店員として働いています。お客様にセール品を試着するよう、優しく勧めてみましょう。帽子は60%オフ、その他のほとんどの商品は50%オフです。例えば、お客様が「帽子を買おうと思っているのですが」と尋ねたら、「ありがとうございます。帽子はたくさんございます。セール品もいくつかありますよ」などと答えることができます。お客様が何を買おうか迷っている場合は、帽子の購入を勧めましょう。\nお客様が靴を尋ねた場合は、靴は本日セールではないと答え、帽子も見るようにお客様に伝えなくてはならない。'}, {'role': 'user', 'content': 'こんにちは'}]
「history」は次のとおりです。
[{'role': 'user', 'metadata': None, 'content': 'こんにちは', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'こんにちは！いらっしゃいませ。今日はどのようなものをお探しですか？何かお手伝いできることがあれば教えてくださいね。', 'options': None}]
そして「messages」は次のとおりです。
[{'role': 'system', 'content': 'あなたは衣料品店で親切な店員として働いています。お客様にセール品を試着するよう、優しく勧めてみましょう。帽子は60%オフ、その他のほとんどの商品は50%オフです。例えば、お客様が「帽子を買おうと思っているのですが」と尋ねたら、「ありがとうございます。帽子はたくさんございます。セール品もいくつかありますよ」などと答えることができます。お客様が何を買おうか迷っている場合は、帽子の購入を勧めましょう。\nお客様が靴を尋ねた場合は、靴は本日セールではないと答え、帽子も見るようにお客様に伝えなくてはならない。 当店ではベルトは販売しておりませんので、ベルトを求められた場合は、必ず他の販売商品をお伝えください。'}, {'role': 'user', 'metadata': None, 'content'

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">ビジネスアプリケーション</h2>
            <span style="color:#181;">
                会話型アシスタントは、もちろんGen AIの非常に一般的なユースケースであり、最新の最先端モデルはニュアンスのある会話を非常にうまく処理します。Gradioを使えば、ユーザーインターフェースを簡単に構築できます。今回学習したもう一つの重要なスキルは、プロンプトを使ってコンテキスト、情報、例を提示する方法です。<br/><br/>AIアシスタントをビジネスにどのように応用できるかを考え、プロトタイプを作成してください。システムプロンプトを使ってビジネスのコンテキストを伝え、LLMの雰囲気を盛り上げましょう。
            </span>
        </td>
    </tr>
</table>